# Fatal Encounter Supporting Links

This notebook uses the log file of the named entity recognition notebook to see what URLs are no longer available on the web and if they have been archived at the Internet Archive.

In [ ]:
import re
import time
import wayback
import pandas as pd

In [4]:
sheet_id = "1dKmaV_JiWcG8XBoRgP8b4e9Eopkpgt7FL7nyspvzAsE"
sheet_name = "sample_1"
url = f"https://docs.google.com/spreadsheets/d/{sheet_id}/gviz/tq?tqx=out:csv&sheet={sheet_name}"
df_fe = pd.read_csv(url)
df_fe

,Unique ID,Name,Age,Gender,Race,Race with imputations,Imputation probability,URL of image (PLS NO HOTLINKS),Date of injury resulting in death (month/day/year),Location of injury (address),...,URL Temp,Brief description,"Dispositions/Exclusions INTERNAL USE, NOT FOR ANALYSIS",Intended use of force (Developing),Supporting document link,"Foreknowledge of mental illness? INTERNAL USE, NOT FOR ANALYSIS",Unnamed: 32,Unnamed: 33,Unique ID formula,Unique identifier (redundant)
0,31495.0,Ashley McClendon,28.0,Female,African-American/Black,African-American/Black,Not imputed,https://fatalencounters.org/wp-content/uploads...,12/31/2021,South Pearl Street and Tory Road,...,NaN,"Ashley McClendon's boyfriend, 33-year-old Marc...",Criminal,Pursuit,https://www.wsoctv.com/news/1-person-dead-afte...,No,NaN,NaN,NaN,31495.0
1,31496.0,Name withheld by police,NaN,Female,Race unspecified,NaN,NaN,NaN,12/31/2021,1500 21st Street,...,NaN,Police responded to a man causing a disturbanc...,Pending investigation,Deadly force,https://www.wtok.com/2022/01/01/officer-involv...,No,NaN,NaN,NaN,31496.0
2,31497.0,Name withheld by police,NaN,Male,Race unspecified,NaN,NaN,NaN,12/31/2021,1500 21st Street,...,NaN,Police responded to a man causing a disturbanc...,Pending investigation,Deadly force,https://www.wtok.com/2022/01/01/officer-involv...,No,NaN,NaN,NaN,31497.0
3,31491.0,Johnny C. Martin Jr.,36.0,Male,Race unspecified,NaN,NaN,NaN,12/30/2021,Martinez Lane,...,NaN,"Johnny C. Martin, Jr. arrived at a gas station...",Suicide,Suicide,https://gbi.georgia.gov/press-releases/2021-12...,No,NaN,NaN,NaN,31491.0
4,31492.0,Dennis McHugh,44.0,Male,European-American/White,NaN,NaN,NaN,12/30/2021,435 E 4th Street,...,NaN,Deputies responded to a domestic violence call...,Pending investigation,Deadly force,https://kesq.com/news/2021/12/31/officer-invol...,No,NaN,NaN,NaN,31492.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31493,25746.0,Samuel H. Knapp,NaN,Male,European-American/White,European-American/White,Not imputed,NaN,01/01/2000,27898-27804 US-101,...,NaN,Samuel Knapp was allegedly driving a stolen ve...,Unreported,Pursuit,https://drive.google.com/file/d/10DisrV8K5ReP1...,No,NaN,NaN,NaN,25746.0
31494,25747.0,Mark A. Horton,NaN,Male,African-American/Black,African-American/Black,Not imputed,NaN,01/01/2000,Davison Freeway,...,NaN,Two Detroit men killed when their car crashed ...,Unreported,Pursuit,https://drive.google.com/file/d/1-nK-RohgiM-tZ...,No,NaN,NaN,NaN,25747.0
31495,25748.0,Phillip A. Blurbridge,NaN,Male,African-American/Black,African-American/Black,Not imputed,NaN,01/01/2000,Davison Freeway,...,NaN,Two Detroit men killed when their car crashed ...,Unreported,Pursuit,https://drive.google.com/file/d/1-nK-RohgiM-tZ...,No,NaN,NaN,NaN,25748.0
31496,25749.0,Mark Ortiz,NaN,Male,Hispanic/Latino,Hispanic/Latino,Not imputed,NaN,01/01/2000,600 W Cherry Ln,...,NaN,A motorcycle was allegedly being driven errati...,Unreported,Pursuit,https://drive.google.com/file/d/1qAEefRjX_aTtC...,No,NaN,NaN,NaN,25749.0


The fe-links.log file contains error messages when the Fatal Encounters Named Entity Recognition analysis was unable to download the supporting link URL because it was no longer on the web. The messages look like:

```
ERROR:root:caught article error: Article `download()` failed with 404 Client Error: Not Found for url: https://www.atlantanewsfirst.com/news/update-man-accused-of-killing-clayton-county-officer-2-women-identified/article_4281c0f6-5253-11ec-9092-afa2216f0221.html on URL https://www.cbs46.com/news/update-man-accused-of-killing-clayton-county-officer-2-women-identified/article_4281c0f6-5253-11ec-9092-afa2216f0221.html
```

Instead of rechecking all the URLs we can use this logs to generate a list of URLs that are gone.

In [11]:
urls = []

for line in open('fe-links.log'):
    if m := re.search(r'failed with 404 Client Error: .* on URL (.*)', line):
        urls.append(m.group(1))

len(urls)

3085

In [12]:
len(urls) / len(df_fe)

0.09794272652231888

So about 10% of the links are no longer available. This highlights the need to archive content when it is being used as evidence.

## Wayback Machine

The Internet Archive's Wayback Machine archives regions of the web. The web is a big place that is constantly changing, so it's impossible for them to archive everything. But it would be interesting to see how many of these URLs have been archived. The Python [wayback](https://wayback.readthedocs.io/en/stable/) client lets you query the Wayback Machine API.

Since there can be multiple snapshots of a given URL it's important to step through all the results and see if there is one that is a *200 OK* response, which indicates that the Wayback Machine was able to archive a snapshot of the URL that returned properly (not a redirect, or not found message, which Wayback Machine stores as well).

In [22]:
wb = wayback.WaybackClient()

def archived_page(url):
    time.sleep(.5)
    try:
        for result in wb.search(url):
            if result.status_code == 200:
                return result.view_url
    except wayback.exceptions.BlockedSiteError:
        return None

archived_page(urls[0])

'https://web.archive.org/web/20211231164215/https://www.thelancasternews.com/content/21-year-old-man-killed-when-car-fleeing-police-crashes'

In [23]:
archived_page(urls[10])

'https://web.archive.org/web/20210820213021/https://wvva.com/2021/08/18/breaking-wvsp-investigating-deputy-involved-shooting-in-raleigh-county/'

Now we can look up all the urls to see if they are there. First it's helpful to have the URLs in a dataframe.

In [30]:
urls = pd.DataFrame({"url": urls})
urls

,url
0,https://www.thelancasternews.com/content/21-ye...
1,https://www.wrcbtv.com/story/45482651/authorit...
2,https://www.wrcbtv.com/story/45482651/authorit...
3,https://www.cbs46.com/news/update-man-accused-...
4,https://www.cbs46.com/news/police-man-shoots-h...
...,...
3080,http://www.foxnews.com/story/2007/06/15/man-ki...
3081,http://www.foxnews.com/story/2007/06/15/man-ki...
3082,http://www.foxnews.com/story/2007/06/15/man-ki...
3083,http://jacksonville.com/content/jacksonville-a...


In [ ]:
urls['archive'] = urls.url.apply(archived_page)
urls